In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies=pd.read_csv(r'/Users/lohitashwa/Desktop/Semester 6/RS/ml-latest-small/movies.csv')
movies=movies.set_index('movieId')
movies

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama


In [3]:
ratings=pd.read_csv(r'/Users/lohitashwa/Desktop/Semester 6/RS/ml-latest-small/ratings.csv')
ratings=ratings.set_index('userId')
ratings

,movieId,rating,timestamp
userId,,,
1,1,4.0,964982703
1,3,4.0,964981247
1,6,4.0,964982224
1,47,5.0,964983815
1,50,5.0,964982931
...,...,...,...
610,166534,4.0,1493848402
610,168248,5.0,1493850091
610,168250,5.0,1494273047


In [4]:
merged = pd.merge(movies, ratings, on='movieId')
merged

,movieId,title,genres,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.5,1305696483
...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,4.0,1537109082
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,3.5,1537109545
100833,193585,Flint (2017),Drama,3.5,1537109805
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,3.5,1537110021


## Getting Genres of all the movies

In [5]:
genres = np.unique(np.concatenate(movies['genres'].str.split('|').values))
genres

array(['(no genres listed)', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western'], dtype='<U18')

## User Profile

In [6]:
def user_profile(active_user,movies,ratings,genres):
    user_profile=ratings.loc[active_user,['movieId','rating']]
    user_item_profile=pd.DataFrame(columns=genres, index=user_profile['movieId'])
    for i, row in movies.loc[user_profile['movieId'].values].iterrows():
        genres = row['genres'].split('|')
        user_item_profile.loc[i, tuple(genres)] = 1
    user_item_profile=user_item_profile.fillna(0)
    user_item_profile = user_item_profile.mul(user_profile['rating'].values, axis=0)
    user_item_profile=user_item_profile.sum()
    return user_item_profile / user_item_profile.sum()
    

## Cosine Similarty

In [7]:
def similarity(active_user,movies,ratings,genres,user_item_profile):
    # droping those movies which active user already rated it 
    movies=movies.drop(ratings.loc[active_user,'movieId'].values)
    # print(movies)
    item_profile=pd.DataFrame(columns=genres, index=movies.index)
    for i, row in movies.iterrows():
        genres = row['genres'].split('|')
        item_profile.loc[i, tuple(genres)] = 1
    item_profile=item_profile.fillna(0)
    # print(item_profile)
    mapped_item_profile=item_profile.mul(user_item_profile.values.T)
    # print(mapped_item_profile)
    return pd.DataFrame(cosine_similarity(user_item_profile.values.T.reshape(1, -1), mapped_item_profile.values), columns=mapped_item_profile.index, index=[active_user])
    # 

## Content Base Recommendation System

In [8]:
def recommendation(active_user,movies,ratings,genres):
    user_item_profile=user_profile(active_user,movies,ratings,genres)
    # print(user_item_profile)
    similarties=similarity(active_user,movies,ratings,genres,user_item_profile)
    similarties=similarties.loc[:, similarties.iloc[0].sort_values(ascending=False).index]
    # recommending 3 movies
    similarties=similarties.iloc[:,:3]
    return movies.loc[similarties.columns,'title']


In [9]:
active_user=int(input("Enter The User Id "))
print(recommendation(active_user,movies,ratings,genres))

Enter The User Id 1
movieId
81132                             Rubber (2010)
117646    Dragonheart 2: A New Beginning (2000)
4956                      Stunt Man, The (1980)
Name: title, dtype: object
